In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
solutions_train = pd.read_excel('./../data/raw/train/solutions.xlsx')
tasks_train = pd.read_excel('./../data/raw/train/tasks.xlsx')
tests_train = pd.read_excel('./../data/raw/train/tests.xlsx')

solutions_test = pd.read_excel('./../data/raw/test/solutions.xlsx')
tasks_test = pd.read_excel('./../data/raw/test/tasks.xlsx')
tests_test = pd.read_excel('./../data/raw/test/tests.xlsx')

In [3]:
df_train = solutions_train.merge(tasks_train.rename(columns={'id': 'task_id'}), on='task_id', how='left')
df_test = solutions_test.merge(tasks_test.rename(columns={'id': 'task_id'}), on='task_id', how='left')

unit_map_train = lambda row: tests_train[tests_train['task_id'] == row.task_id][['type', 'input', 'output']] \
    .rename(columns={'type': 'Тип теста', 'input': 'Вход', 'output': 'Верный вывод программы'}) \
    .to_string(index=False)
unit_map_test = lambda row: tests_test[tests_test['task_id'] == row.task_id][['type', 'input', 'output']] \
    .rename(columns={'type': 'Тип теста', 'input': 'Вход', 'output': 'Верный вывод программы'}) \
    .to_string(index=False)

df_train['unit_true'] = df_train.apply(unit_map_train, axis=1)
df_test['unit_true'] = df_test.apply(unit_map_test, axis=1)
df_test['author_comment'] = ''

system = 'Вы учитель и вы должны на русском языке давать подсказки без прямых исправлений кода, поддерживая формальный и дружелюбный стиль общения'
df_train['system'] = df_test['system'] = system

df_train_train, df_train_test = train_test_split(df_train, test_size=0.1, random_state=42)
df_train_train.to_csv('./../data/raw/train/data_full_train.csv')
df_train_test.to_csv('./../data/raw/train/data_full_test.csv')
df_test.to_csv('./../data/raw/test/data_full.csv')

df_train.head(3)

,id,task_id,student_solution,author_comment,author_comment_embedding,level,description,author_solution,unit_true,system
0,13,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.2475823611021042 -1.3317935466766357 0.2535...,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Тип теста Вход Верный вывод программы\n ...,Вы учитель и вы должны на русском языке давать...
1,14,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.25970256328582764 -1.4550446271896362 0.333...,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Тип теста Вход Верный вывод программы\n ...,Вы учитель и вы должны на русском языке давать...
2,15,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.28293243050575256 -1.4774413108825684 0.243...,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Тип теста Вход Верный вывод программы\n ...,Вы учитель и вы должны на русском языке давать...


In [4]:
row = df_train.iloc[0]
prompt = '''### INSTRUCTION:
{}

### ОСНОВНАЯ ЗАДАЧА ДЛЯ СТУДЕНТА:
{}

### ВЕРНОЕ РЕШЕНИЕ АВТОРА:
```python
{}
```

### ВЕРНЫЕ ОТВЕТЫ НА UNIT ТЕСТЫ:
{}

### НЕВЕРНОЕ РЕШЕНИЕ СТУДЕНТА:
```python
{}
```

### КОММЕНТАРИИ К НЕВЕРНОМУ РЕШЕНИЮ:
{}'''
model_input = prompt.format(*row[['system', 'description', 'author_solution', 'unit_true', 'student_solution', 'author_comment']])
print(model_input)

### INSTRUCTION:
Вы учитель и вы должны на русском языке давать подсказки без прямых исправлений кода, поддерживая формальный и дружелюбный стиль общения

### ОСНОВНАЯ ЗАДАЧА ДЛЯ СТУДЕНТА:
Реализуйте программу, которая проверит, что цвет используется только в проекте по созданию логотипа, но не в проекте по созданию дизайна сайта:

Даны два списка logo_project и cite_project с кодами используемых цветов (строки).
В переменную color считывается код цвета (строка). Этот код уже написан.
Программа должна проверять, что код цвета color есть только в списке logo_project, и если да, то печатать True. 
В остальных случаях программа печатает False. 

### ВЕРНОЕ РЕШЕНИЕ АВТОРА:
```python
logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']
cite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']

color = input()

if color in logo_project and not(color in cite_project):
    print(True)
else:
    print(False)
```

### ВЕРНЫЕ ОТВЕТЫ НА UNIT ТЕС

In [5]:
from unsloth import FastLanguageModel
import torch
import os, dotenv
dotenv.load_dotenv()
max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [6]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-2-9b-it",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...",
)

==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.551 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post4 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [8]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    texts = []
    for system, description, author_solution, unit_true, student_solution, author_comment in zip(
            examples['system'], 
            examples['description'], 
            examples['author_solution'], 
            examples['unit_true'], 
            examples['student_solution'],
            examples['author_comment']):
        text = prompt.format(system, description, author_solution, unit_true, student_solution, author_comment) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

from datasets import load_dataset
data_files = {
    'train': './../data/raw/train/data_full_train.csv',
    'test': './../data/raw/train/data_full_test.csv'
}
dataset = load_dataset('csv', data_files=data_files)
dataset = dataset.map(formatting_prompts_func, batched = True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/312 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['test'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        num_train_epochs=12,
        per_device_train_batch_size = 4,
        per_device_eval_batch_size = 2,
        gradient_accumulation_steps = 2,
        warmup_steps = 10,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0025,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",

        load_best_model_at_end = True,
        greater_is_better=False,
        eval_steps=9,
        save_steps=9,
        eval_strategy="steps",
        save_strategy="steps",
    ),
)

Map (num_proc=2):   0%|          | 0/312 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/35 [00:00<?, ? examples/s]

In [10]:
trainer_stats = trainer.train()
model = trainer.model

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 312 | Num Epochs = 12
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 468
 "-____-"     Number of trainable parameters = 54,018,048


  0%|          | 0/468 [00:00<?, ?it/s]

{'loss': 1.6202, 'grad_norm': 5.124875545501709, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 1.6549, 'grad_norm': 4.661335468292236, 'learning_rate': 1e-05, 'epoch': 0.05}
{'loss': 1.7256, 'grad_norm': 4.582201957702637, 'learning_rate': 1.5e-05, 'epoch': 0.08}
{'loss': 1.8302, 'grad_norm': 4.539109706878662, 'learning_rate': 2e-05, 'epoch': 0.1}
{'loss': 1.4906, 'grad_norm': 3.710376739501953, 'learning_rate': 2.5e-05, 'epoch': 0.13}
{'loss': 1.3346, 'grad_norm': 2.391968250274658, 'learning_rate': 3e-05, 'epoch': 0.15}
{'loss': 1.2545, 'grad_norm': 4.084812164306641, 'learning_rate': 3.5e-05, 'epoch': 0.18}
{'loss': 1.14, 'grad_norm': 2.6466970443725586, 'learning_rate': 4e-05, 'epoch': 0.21}
{'loss': 1.1816, 'grad_norm': 2.5857391357421875, 'learning_rate': 4.5e-05, 'epoch': 0.23}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 1.0954420566558838, 'eval_runtime': 12.6418, 'eval_samples_per_second': 2.769, 'eval_steps_per_second': 1.424, 'epoch': 0.23}
{'loss': 1.0171, 'grad_norm': 2.5206782817840576, 'learning_rate': 5e-05, 'epoch': 0.26}
{'loss': 1.0224, 'grad_norm': 2.501950263977051, 'learning_rate': 4.989082969432315e-05, 'epoch': 0.28}
{'loss': 0.9146, 'grad_norm': 1.8020570278167725, 'learning_rate': 4.978165938864629e-05, 'epoch': 0.31}
{'loss': 0.9086, 'grad_norm': 2.130194664001465, 'learning_rate': 4.967248908296944e-05, 'epoch': 0.33}
{'loss': 0.6462, 'grad_norm': 1.4950077533721924, 'learning_rate': 4.956331877729258e-05, 'epoch': 0.36}
{'loss': 0.7212, 'grad_norm': 1.6372754573822021, 'learning_rate': 4.945414847161572e-05, 'epoch': 0.38}
{'loss': 0.4618, 'grad_norm': 1.7959424257278442, 'learning_rate': 4.9344978165938866e-05, 'epoch': 0.41}
{'loss': 0.4994, 'grad_norm': 2.0408594608306885, 'learning_rate': 4.923580786026201e-05, 'epoch': 0.44}
{'loss': 0.6286, 'grad_norm': 1.81146

...

{'eval_loss': 0.03691587597131729, 'eval_runtime': 7.9068, 'eval_samples_per_second': 4.427, 'eval_steps_per_second': 2.277, 'epoch': 11.54}
{'loss': 0.0069, 'grad_norm': 0.12172950059175491, 'learning_rate': 1.8558951965065505e-06, 'epoch': 11.56}
{'loss': 0.0081, 'grad_norm': 0.13706372678279877, 'learning_rate': 1.7467248908296944e-06, 'epoch': 11.59}
{'loss': 0.0067, 'grad_norm': 0.11949306726455688, 'learning_rate': 1.6375545851528385e-06, 'epoch': 11.62}
{'loss': 0.0074, 'grad_norm': 0.14043323695659637, 'learning_rate': 1.5283842794759824e-06, 'epoch': 11.64}
{'loss': 0.0077, 'grad_norm': 0.14101247489452362, 'learning_rate': 1.4192139737991267e-06, 'epoch': 11.67}
{'loss': 0.007, 'grad_norm': 0.11888639628887177, 'learning_rate': 1.3100436681222706e-06, 'epoch': 11.69}
{'loss': 0.0077, 'grad_norm': 0.12010487914085388, 'learning_rate': 1.200873362445415e-06, 'epoch': 11.72}
{'loss': 0.0065, 'grad_norm': 0.11477198451757431, 'learning_rate': 1.0917030567685589e-06, 'epoch': 11.7

  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 0.03712288662791252, 'eval_runtime': 7.9431, 'eval_samples_per_second': 4.406, 'eval_steps_per_second': 2.266, 'epoch': 11.77}
{'loss': 0.0048, 'grad_norm': 0.0814783051609993, 'learning_rate': 8.733624454148472e-07, 'epoch': 11.79}
{'loss': 0.0065, 'grad_norm': 0.10069309175014496, 'learning_rate': 7.641921397379912e-07, 'epoch': 11.82}
{'loss': 0.006, 'grad_norm': 0.09762673079967499, 'learning_rate': 6.550218340611353e-07, 'epoch': 11.85}
{'loss': 0.0064, 'grad_norm': 0.10766149312257767, 'learning_rate': 5.458515283842794e-07, 'epoch': 11.87}
{'loss': 0.0077, 'grad_norm': 0.1331329196691513, 'learning_rate': 4.366812227074236e-07, 'epoch': 11.9}
{'loss': 0.0061, 'grad_norm': 0.1034250557422638, 'learning_rate': 3.2751091703056766e-07, 'epoch': 11.92}
{'loss': 0.0065, 'grad_norm': 0.1075541079044342, 'learning_rate': 2.183406113537118e-07, 'epoch': 11.95}
{'loss': 0.0073, 'grad_norm': 0.23750033974647522, 'learning_rate': 1.091703056768559e-07, 'epoch': 11.97}
{'loss':

  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 0.0371207669377327, 'eval_runtime': 7.9475, 'eval_samples_per_second': 4.404, 'eval_steps_per_second': 2.265, 'epoch': 12.0}
{'train_runtime': 3363.6796, 'train_samples_per_second': 1.113, 'train_steps_per_second': 0.139, 'train_loss': 0.0675453081056396, 'epoch': 12.0}


In [21]:
FastLanguageModel.for_inference(model)
ix = 41
inputs = tokenizer(
[
    prompt.format(
        df_train.iloc[ix]['system'],
        df_train.iloc[ix]['description'],
        df_train.iloc[ix]['author_solution'],
        df_train.iloc[ix]['unit_true'],
        df_train.iloc[ix]['student_solution'],
        '',
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### INSTRUCTION:
Вы учитель и вы должны на русском языке давать подсказки без прямых исправлений кода, поддерживая формальный и дружелюбный стиль общения

### ОСНОВНАЯ ЗАДАЧА ДЛЯ СТУДЕНТА:
Реализуйте программу, которая напечатает хэштеги из фраз для лендинга сайта: 

В переменную logo считывается текст лендинга. Этот код уже написан.
Программа проверяет, есть ли в этом тексте хэштеги.  Хэштегами можно считать все элементы, которые начинаются со знака "#" и которые не состоят только из цифр (не включая знак "#"). Гарантируется, что хэштеги отделены от другого текста пробелом. 
В конце программа печатает все хэштеги через запятую с пробелом. Хэштеги выводятся в том же порядке, в котором они идут в тексте.
Если знаков хэштегов в тексте нет, то программа ничего не печатает.

### ВЕРНОЕ РЕШЕНИЕ АВТОРА:
```python
logo = input()

res = []
for info in logo.split():
    if info.startswith('#') and info[1:].isdigit() == False:      
        res.append(info)
        
print(*res, sep=', ')
```

##

In [22]:
print(df_train.iloc[ix]['author_comment'])

Ошибка в открытых и скрытых тестах. 

Ваш код не проверяет все условия задания. Например, он некорректно выполняет условие "В конце программа печатает все хэштеги через запятую с пробелом". Дополните функцию print(), чтобы выполнить данное условие.


In [17]:
model.save_pretrained("./../gemma-2-9b")
tokenizer.save_pretrained("./../gemma-2-9b")

('gemma-2-9b/tokenizer_config.json',
 'gemma-2-9b/special_tokens_map.json',
 'gemma-2-9b/tokenizer.model',
 'gemma-2-9b/added_tokens.json',
 'gemma-2-9b/tokenizer.json')